# 1. Import Python Packages

In [ ]:
import sys, os
sys.path.insert(0,'/global/homes/b/bpb/metatlas/' )
sys.path.insert(1,'/global/project/projectdirs/metatlas/anaconda/lib/python2.7/site-packages' )
# sys.path.insert(1,'/global/project/projectdirs/metatlas/projects/metatlas' )

from metatlas.helpers import dill2plots as dp

from metatlas.helpers import metatlas_get_data_helper_fun as ma_data
from metatlas import metatlas_objects as metob
from metatlas.helpers import rt_corrector as rt_corrector
from ipywidgets import interact, interactive, fixed

import ipywidgets as widgets
from IPython.display import display

%matplotlib notebook

# 2. Register LCMS Runs into categorical groups.

* ### Select MetAtlas LCMS Runs by experiment and filename.  

In [ ]:
f = dp.interact_get_metatlas_files()

* ### Create a "File-Info" sheet from the selected files.  
This sheet needs to be downloaded and filled in.  The "File-Info" sheet is the exchange format we use to define the grouping membership for LCMS runs.


In [ ]:
dp.make_empty_fileinfo_sheet('/global/homes/b/bpb/Downloads/empty_finfo_for_jimmy.tab',f.files)

* ### Create metatlas groups from filled in file-info sheet
Defining groups of files allows for the selection of sets of LCMS runs by specifying the group names.  In addition,  the group membership is preserved in the exported metatlas datasets; so the application of statistical methods based on grouped data is straightforward.

Your filled in sheet will look something like this:
<style type="text/css">
	table.tableizer-table {
		font-size: 12px;
		border: 1px solid #CCC; 
		font-family: Arial, Helvetica, sans-serif;
	} 
	.tableizer-table td {
		padding: 4px;
		margin: 3px;
		border: 1px solid #CCC;
	}
	.tableizer-table th {
		background-color: #104E8B; 
		color: #FFF;
		font-weight: bold;
	}
</style>
<table class="tableizer-table">
<thead><tr class="tableizer-firstrow"><th>mzml_file</th><th>group</th><th>description</th></tr></thead><tbody>
 <tr><td>.../20160531_KBL_violacein_cells_384_final/20160531_C18_ACN_POS_MSMS_KBL_Qex_A_10_Run413.mzML</td><td>20160531_KBL_C18_Vio_cells_384_Quad_1</td><td>&nbsp;</td></tr>
 <tr><td>.../20160531_KBL_violacein_cells_384_final/20160531_C18_ACN_POS_MSMS_KBL_Qex_A_11_Run415.mzML</td><td>20160531_KBL_C18_Vio_cells_384_Quad_1</td><td>&nbsp;</td></tr>
 <tr><td>.../20160531_KBL_violacein_cells_384_final/20160531_C18_ACN_POS_MSMS_KBL_Qex_A_12_Run417.mzML</td><td>20160531_KBL_C18_Vio_cells_384_Quad_1</td><td>&nbsp;</td></tr>
 <tr><td>.../20160531_KBL_violacein_cells_384_final/20160531_C18_ACN_POS_MSMS_KBL_Qex_A_1_Run395.mzML</td><td>20160531_KBL_C18_Vio_cells_384_Quad_2</td><td>&nbsp;</td></tr>
 <tr><td>.../20160531_KBL_violacein_cells_384_final/20160531_C18_ACN_POS_MSMS_KBL_Qex_A_2_Run397.mzML</td><td>20160531_KBL_C18_Vio_cells_384_Quad_2</td><td>&nbsp;</td></tr>
 <tr><td>.../20160531_KBL_violacein_cells_384_final/20160531_C18_ACN_POS_MSMS_KBL_Qex_A_3_Run399.mzML</td><td>20160531_KBL_C18_Vio_cells_384_Quad_2</td><td></td></tr>
</tbody></table>

A text description of each group is an optional field.  These can be a few, short sentences that describe each group.

In [ ]:
g = dp.make_groups_from_fileinfo_sheet('/global/homes/b/bpb/Downloads/finfo_jimmy_filled_in.csv',
                                       filetype='csv',
                                       store=False)

View the list of metatlas objects using "to_dataframe"

In [ ]:
metob.to_dataframe(g).head()

# 3. Create a new Atlas
* ### From Google Doc Reference Data

In [ ]:
dp = reload(dp)
atlas_sheet = dp.get_formatted_atlas_from_google_sheet(polarity='POS',
                                          method='QE_HILIC',
                                          mz_tolerance=10)

In [ ]:
atlas_sheet.tail()

* ### From a pandas dataframe

In [ ]:
dp = reload(dp)
names = dp.make_atlas_from_spreadsheet('',
                                       'QE_HILIC_POS_ISTD_AND_COMMON_METABOLITES',
                                       filetype='dataframe',
                                       sheetname='',
                                       polarity = 'positive',
                                       store=True,
                                       dataframe = atlas_sheet,
                                      mz_tolerance = 10)

* ### From a spreadsheet
This is by far the most common way to create a new Atlas in Metabolite Atlas.  The columns the sheet must be exactly as what is seen here.  In cases where there isn't a compound in the database, the "label" field below is used.  Here is an example of what a sheet could look like.

<style type="text/css">
	table.tableizer-table {
		font-size: 12px;
		border: 1px solid #CCC; 
		font-family: Arial, Helvetica, sans-serif;
	} 
	.tableizer-table td {
		padding: 4px;
		margin: 3px;
		border: 1px solid #CCC;
	}
	.tableizer-table th {
		background-color: #104E8B; 
		color: #FFF;
		font-weight: bold;
	}
</style>
<table class="tableizer-table">
<thead><tr class="tableizer-firstrow"><th>label</th><th>rt_min</th><th>rt_max</th><th>rt_peak</th><th>mz</th><th>mz_tolerance</th><th>inchi_key</th></tr></thead><tbody>
 <tr><td>violacein </td><td>4.2</td><td>4.4</td><td>4.3</td><td>344.1036913</td><td>5</td><td>XAPNKXIRQFHCHN-QGOAFFKASA-N</td></tr>
 <tr><td>deoxyviolacein (iso1 - main)</td><td>4.75</td><td>4.9</td><td>4.8</td><td>328.1087767</td><td>5</td><td>OJUJNNKCVPCATE-QGOAFFKASA-N</td></tr>
 <tr><td>tryptophan</td><td>2.3</td><td>2.45</td><td>2.36</td><td>205.0978776</td><td>5</td><td>QIVBCDIJIAJPQS-VIFPVBQESA-N</td></tr>
 <tr><td>deoxychromoviridans</td><td>5.4</td><td>6</td><td>5.75</td><td>605.244821</td><td>5</td><td>&nbsp;</td></tr>
 <tr><td>chromoviridans</td><td>5.15</td><td>5.5</td><td>5.3</td><td>621.239736</td><td>5</td><td>&nbsp;</td></tr>
 <tr><td>ABMBA</td><td>4.72</td><td>4.88</td><td>4.8</td><td>229.9811</td><td>5</td><td>LCMZECCEEOQWLQ-UHFFFAOYSA-N</td></tr>
</tbody></table>

These tables can be csv or tab delimited text or excel spreadsheets.

There is a <a href = 'https://drive.google.com/open?id=0BweRoDuGcsLBNkYtQUpjQ0ktZDg'> lookup table here </a> of all compounds to get the inchi_key.

For old MetAtlas atlases, you can use Excel's "vlookup" function along with <a href = 'https://drive.google.com/open?id=0BweRoDuGcsLBQUxrRjgtbjhnSDg'> this lookup table </a> to map the old names to valid inchi keys.

```
=VLOOKUP(H2,$A:$B,2,0) where $A:$B are columns containing name and inchi-key
```

This is a <a href = 'https://drive.google.com/open?id=0BweRoDuGcsLBaThjcEZuSjh2dXM'> link </a> to all the old compound identifications that were in the database prior to the refactoring in Mid June, 2016.


In [ ]:
dp = reload(dp)
names = dp.make_atlas_from_spreadsheet('/global/homes/b/bpb/Downloads/KZ_Avena_Uptake_atlases_and_group_2/2016019_metalas_KZ_Ave_uptake_qxqct_hilic_updated_pos_V2.txt',
                                       '2016019_metatlas_KZ_Ave_uptake_qxqct_hilic_updated_pos_V2',
                                       filetype='tab',
                                       sheetname='',
                                       polarity = 'positive',
                                       store=True,
                                      mz_tolerance = 20)

# 0 73 2016019_metalas_KZ_Ave_uptake_qxqct_hilic_updated_pos_V2 2016-07-26 17:31:26
# 1 69 201500826_KZ_Ave_library_qxqct_hilic_pos_V2 2016-07-26 18:17:32
# 2 93 201500826_KZ_Ave_library_qxqct_hilic_neg_V2 2016-07-26 18:14:40
# 3 106 2016019_metatlas_KZ_Ave_uptake_qxqct_hilic_updated_neg_V2 2016-07-26 18:02:34
        

group1:
groups start with KZ_RootExu_
and
end with pos or neg

pos atlas
201500826_KZ_Ave_library_qxqct_hilic_pos_V2
output_filename = '/global/homes/b/bpb/Downloads/KZ_Avena_Exudate_atlases_and groups_1/pos_data.pkl'

neg atlas
201500826_KZ_Ave_library_qxqct_hilic_neg_V2
output_filename = '/global/homes/b/bpb/Downloads/KZ_Avena_Exudate_atlases_and groups_1/neg_data.pkl'



group2
neg atlas
2016019_metatlas_KZ_Ave_uptake_qxqct_hilic_updated_neg_V2
output_filename = '/global/homes/b/bpb/Downloads/KZ_Avena_Uptake_atlases_and_group_2/neg_data.pkl'


pos atlas
2016019_metalas_KZ_Ave_uptake_qxqct_hilic_updated_pos_V2
output_filename = '/global/homes/b/bpb/Downloads/KZ_Avena_Uptake_atlases_and_group_2/pos_data.pkl'


groups all start with:
20160119_KZ
end with pos or neg

# 4. Select groups of files to operate on

In [ ]:
dp = reload(dp)
groups = dp.select_groups_for_analysis(name = '%20160801_KBL_C18_MO_highmz%',
                                       most_recent = True,
                                       remove_empty = True,
                                       include_list = [], exclude_list = ['NEG','FPS'])#QC','Blank'])

# 5. Select Atlas to use
* ### Select by Atlas name

In [ ]:
atlas = dp.get_metatlas_atlas(name='%20160%_POS_MO_HEfungus%',do_print = True,most_recent=True)

* ### A list of atlases is returned by the cell above.  
You must run the following cell to specify which Atlas you want to continue your analysis with (Even if only a single atlas is returned).

In [ ]:
myAtlas = atlas[4]
print myAtlas.name
print myAtlas.username

Export Atlas to a Spreadsheet

In [ ]:
atlas_identifications = dp.export_atlas_to_spreadsheet(myAtlas,'/global/homes/b/bpb/Downloads/20160823_POS_MO_HEfungusonly_V21.csv')


* ### Search for Atlases and display their contents

In [ ]:
rt_corrector.display_atlases()

# 6. Get Data and Save it

### Old faithful, slow data getting script

In [ ]:
output_filename = '/global/homes/b/bpb/Downloads/20160531_KBL_C18_Vio_cells_384_Q_1_to_4.pkl'
data = dp.get_data_for_groups_and_atlas(groups,myAtlas,output_filename)

### Prototype of much faster data getting script

```
Will eventually replace:
dp.get_data_for_groups_and_atlas()
```

In [ ]:
import time
import dill
import copy
import numpy as np
import multiprocessing as mp
from random import randint
from time import sleep

ma_data = reload(ma_data)

def get_data_for_atlas_df_and_file(input_tuple):
    my_file = input_tuple[0]
    atlas_df = input_tuple[1]
    t = float(randint(0, 500)) / 1000
    sleep(t)
    df_container = ma_data.df_container_from_metatlas_file(my_file)
    df_container = ma_data.remove_ms1_data_not_in_atlas(atlas_df,df_container)

    dict_ms1_summary,dict_eic,dict_ms2 = ma_data.get_data_for_atlas_and_lcmsrun(atlas_df,df_container)
    row = []
    for i in range(len(dict_eic)):
        result = {}
        result['atlas_name'] = myAtlas.name
        result['atlas_unique_id'] = myAtlas.unique_id
        result['lcmsrun'] = my_file
        result['group'] = my_group
        temp_compound = copy.deepcopy(myAtlas.compound_identifications[i])
        result['identification'] = temp_compound
        result['data'] = {}
        result['data']['eic'] = dict_eic[i]
        result['data']['ms1_summary'] = dict_ms1_summary[i]
        result['data']['msms'] = {}
        if dict_ms2[i]['mz']:
            for k in dict_ms2[0].keys():
                dict_ms2[i][k] = np.asarray(dict_ms2[i][k])
#                 if temp_compound.mz_references[0].observed_polarity == 'positive':
#                     dict_ms2[i]['polarity'] = dict_ms2[i]['mz'] * 0.0 + 1.0
#                 else:
#                     dict_ms2[i]['polarity'] = dict_ms2[i]['mz'] * 0.0
            result['data']['msms']['data'] = dict_ms2[i]
        else:
            result['data']['msms']['data'] = []
        row.append(result)
#     print my_file.name,time.time() - t0
    return row
#         data.append(row)
        
atlas_df = ma_data.make_atlas_df(myAtlas)
all_files = []
data = []
t0 = time.time()

for my_group in groups:
    for my_file in my_group.items:
        print my_file.name
        all_files.append((my_file,atlas_df))
        data.append(get_data_for_atlas_df_and_file((my_file,atlas_df)))

print time.time() - t0


In [ ]:
# pool = mp.Pool(processes=5)
# t0 = time.time()
# data = pool.map(get_data_for_atlas_df_and_file, all_files)
# print time.time() - t0

In [ ]:
output_filename = '/global/homes/b/bpb/Downloads/20160818_POS_MO_HEfungusonly_V1.pkl'
with open(output_filename,'w') as f:
    dill.dump(data,f)

# 7. Adjust Retention Times.  

* ### RT Adjust: Cell 1:
Load a pre-existing pickle file.  This contains the chromatograms and links to an Atlas in the database.  Show grid of compounds.  Select one row from the table before running cell 2.

In [ ]:
dp = reload(dp)
data,compound_grid = dp.show_compound_grid(input_dataset = metatlas_dataset)#,input_fname = '/global/homes/b/bpb/Downloads/20160531_KBL_C18_Vio_cells_384_Q_1_to_4.pkl')

* ### RT Adjust Cell 2:
Show RT adjustment interface for compound selected in table above.

In [ ]:
dp = reload(dp)
dp.adjust_rt_for_selected_compound(metatlas_dataset,compound_grid,compound_idx = 23,include_lcmsruns=[],exclude_lcmsruns=[], width = 14,y_scale='linear',alpha = 0.02,y_max = 'auto')

# 8. Make Supplementary Tables

* ### Define a pre-existing datafile and location to save plots

In [ ]:
# my_file = '/global/homes/b/bpb/Downloads/KZ_Avena_Exudate_atlases_and groups_1/neg_data.pkl'
my_file = '/global/homes/b/bpb/Downloads/20160531_KBL_C18_Vio_cells_384_Q_1_to_4.pkl'
# my_file = '/global/homes/b/bpb/Downloads/20160818_POS_MO_HEfungusonly_V1.pkl'
# plot_location_label = '/global/homes/b/bpb/Downloads/20160818_POS_MO_HEfungusonly_V1/'
plot_location_label = '/global/homes/b/bpb/Downloads/mp_plots_test/'
if not os.path.exists(plot_location_label):
    os.makedirs(plot_location_label)

* ### Load a pre-existing metatlas dataset  
Although all the functions below can take a path to a dataset, it will save time to load it once and pass the dataset into each function.

In [ ]:
metatlas_dataset = ma_data.get_dill_data(my_file)

* ### Dataframes and spreadsheets

In [ ]:
dp = reload(dp)
# atlas_identifications = dp.export_atlas_to_spreadsheet(myAtlas,'%s/sheets/%s.csv'%(plot_location_label,myAtlas.name))
peak_height = dp.make_output_dataframe(input_fname = '',input_dataset = metatlas_dataset,include_lcmsruns = [],exclude_lcmsruns = [], fieldname='peak_height' , output_loc=plot_location_label+'/sheets')
peak_area = dp.make_output_dataframe(input_fname = my_file,input_dataset = metatlas_dataset,include_lcmsruns = [],exclude_lcmsruns = [], fieldname='peak_area' , output_loc=plot_location_label+'/sheets')
mz_peak = dp.make_output_dataframe(input_fname = my_file,input_dataset = metatlas_dataset,include_lcmsruns = [],exclude_lcmsruns = [], fieldname='mz_peak' , output_loc=plot_location_label+'/sheets')
rt_peak = dp.make_output_dataframe(input_fname = my_file, input_dataset = metatlas_dataset,include_lcmsruns = [],exclude_lcmsruns = [],fieldname='rt_peak' , output_loc=plot_location_label+'/sheets')
mz_centroid = dp.make_output_dataframe(input_fname = my_file,input_dataset = metatlas_dataset,include_lcmsruns = [],exclude_lcmsruns = [], fieldname='mz_centroid' , output_loc=plot_location_label+'/sheets')
rt_centroid = dp.make_output_dataframe(input_fname = my_file,input_dataset = metatlas_dataset,include_lcmsruns = [],exclude_lcmsruns = [], fieldname='rt_centroid' , output_loc=plot_location_label+'/sheets')

* ### Error bar

In [ ]:
dp = reload(dp)
dp.plot_errorbar_plots(peak_height, output_loc=plot_location_label+'/error_bar_peak_height')

* ### Chromatograms

### run this block to define parameters needed for chromatogram plotting

In [ ]:
import pickle
#TODO: make widget that makes yaml file

info = dict(path = '/global/homes/b/bpb/metatlas/', # path to the metatlas package
            path_idx = 0, # the index where you want to insert the path in the python path
            nCols= 12,
            share_y= True,
            pkl_file= my_file,
            output_dir= os.path.join(plot_location_label,'chromatograms'),
            processes= 20,
            plot_types= 'compounds') #plot_types choices: 'files', 'compounds', 'both'

pickle.dump(info, open("chromatogram_temp_file.p", "wb"))

### execute the plotting command using the multi-processing package

In [ ]:
%run /global/homes/b/bpb/metatlas/metatlas/helpers/chromatograms_mp_plots.py chromatogram_temp_file.p

* ### Identification Figures

In [ ]:
# dp = reload(dp)
%matplotlib inline

dp.make_identification_figure(input_dataset = metatlas_dataset, input_fname = my_file, include_lcmsruns = [],exclude_lcmsruns = ['RootCass','QC','Blank','blank'], output_loc=plot_location_label+'/identification')

# Batch running for generating identification figures

In [ ]:
%matplotlib inline
dp = reload(dp)
pickles = ['/global/homes/b/bpb/Downloads/KZ_Avena_Exudate_atlases_and_groups_1/neg_data.pkl',
          '/global/homes/b/bpb/Downloads/KZ_Avena_Exudate_atlases_and_groups_1/pos_data.pkl',
'/global/homes/b/bpb/Downloads/KZ_Avena_Uptake_atlases_and_group_2/pos_data.pkl',
'/global/homes/b/bpb/Downloads/KZ_Avena_Uptake_atlases_and_group_2/neg_data.pkl']
for p in pickles:
    plot_location_label = p.split('.')[0]+'/'
    print plot_location_label
    if not os.path.exists(plot_location_label):
        os.makedirs(plot_location_label)
    metatlas_dataset = ma_data.get_dill_data(p)
    dp.make_identification_figure(input_dataset = metatlas_dataset, input_fname = p, include_lcmsruns = [],exclude_lcmsruns = ['RootCass','QC','Blank','blank'], output_loc=plot_location_label+'/identification')